In [23]:
import pandas as pd
import numpy as np

In [24]:
from sklearn.metrics import precision_score, accuracy_score
from sklearn.model_selection import GroupKFold
from catboost import CatBoostClassifier

# Load and prepare data

In [25]:
df = pd.read_csv("C:\\Users\\Egor\\Projects\\Ranking_vacancy\\Ml_Hackaton_28-31\\data\\df_all_embeddings.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df.head(3)

,uuid,answer,vacancy_uuid,education,average_workout,languages,text_vacancy,text_resume,vac_emb_0,vac_emb_1,...,res_emb_302,res_emb_303,res_emb_304,res_emb_305,res_emb_306,res_emb_307,res_emb_308,res_emb_309,res_emb_310,res_emb_311
0,8c8cf797-2c6b-3f4b-b28b-20d57bd88b82,1,779f3a59-206a-3241-adc4-d7db504f960b,1.0,1171.000000,0,Java разработчик команда Инвестиции Описание М...,"nanJava, Java Spring Framework, Hibernate, Git...",0.09377,0.035561,...,0.009770,-0.039553,0.027861,0.018131,-0.000193,-0.003084,-0.054248,0.064520,0.042438,-0.035391
1,74221d62-5ea6-3a68-8849-25acd97e208b,1,779f3a59-206a-3241-adc4-d7db504f960b,1.0,1548.666667,1,Java разработчик команда Инвестиции Описание М...,"nanMATLAB, Java, Android SDK, Design Patterns,...",0.09377,0.035561,...,0.006789,0.054488,0.013807,-0.018607,-0.013298,-0.012359,-0.044851,0.007465,0.053414,-0.061761
2,9d7eae36-11f8-3cac-9cb2-4cb0ff9d0ae7,1,779f3a59-206a-3241-adc4-d7db504f960b,1.0,1197.000000,0,Java разработчик команда Инвестиции Описание М...,"Усидчивый и терпеливый программист, обладающи...",0.09377,0.035561,...,0.046191,-0.023448,0.039488,0.017030,-0.005027,0.020015,-0.026526,0.084943,0.063039,-0.035794


In [26]:
# load embeddings
vacancy_emb = pd.read_csv("C:\\Users\\Egor\\Projects\\Ranking_vacancy\\Ml_Hackaton_28-31\\data\\vacancy_embeddings.csv")
resume_emb = pd.read_csv("C:\\Users\\Egor\\Projects\\Ranking_vacancy\\Ml_Hackaton_28-31\\data\\resume_embeddings.csv")

# process embeddings for catboost
df["vacancy_emb"] = vacancy_emb.apply(lambda x: np.array(x), axis=1)
df["resume_emb"] = resume_emb.apply(lambda x: np.array(x), axis=1)

In [27]:
df.head(3)

,uuid,answer,vacancy_uuid,education,average_workout,languages,text_vacancy,text_resume,vac_emb_0,vac_emb_1,...,res_emb_304,res_emb_305,res_emb_306,res_emb_307,res_emb_308,res_emb_309,res_emb_310,res_emb_311,vacancy_emb,resume_emb
0,8c8cf797-2c6b-3f4b-b28b-20d57bd88b82,1,779f3a59-206a-3241-adc4-d7db504f960b,1.0,1171.000000,0,Java разработчик команда Инвестиции Описание М...,"nanJava, Java Spring Framework, Hibernate, Git...",0.09377,0.035561,...,0.027861,0.018131,-0.000193,-0.003084,-0.054248,0.064520,0.042438,-0.035391,"[0.09376956, 0.035560623, 0.0075116404, -0.082...","[0.03439939, -0.0072224373, 0.08023889, -0.063..."
1,74221d62-5ea6-3a68-8849-25acd97e208b,1,779f3a59-206a-3241-adc4-d7db504f960b,1.0,1548.666667,1,Java разработчик команда Инвестиции Описание М...,"nanMATLAB, Java, Android SDK, Design Patterns,...",0.09377,0.035561,...,0.013807,-0.018607,-0.013298,-0.012359,-0.044851,0.007465,0.053414,-0.061761,"[0.09376956, 0.035560623, 0.0075116404, -0.082...","[-0.0025297515, -0.025079247, 0.06452319, -0.0..."
2,9d7eae36-11f8-3cac-9cb2-4cb0ff9d0ae7,1,779f3a59-206a-3241-adc4-d7db504f960b,1.0,1197.000000,0,Java разработчик команда Инвестиции Описание М...,"Усидчивый и терпеливый программист, обладающи...",0.09377,0.035561,...,0.039488,0.017030,-0.005027,0.020015,-0.026526,0.084943,0.063039,-0.035794,"[0.09376956, 0.035560623, 0.0075116404, -0.082...","[0.003054467, -0.024304809, 0.050865255, -0.07..."


# Train model

In [29]:
# define numerical features 
cols_to_fit = ["education", "average_workout", "languages"]
# define embeddings features
cols_embed = ["vacancy_emb", "resume_emb"]

gkf = GroupKFold(n_splits=5)
loss_hist = []

# cross-validation spliting folds by vacancy
for i, (train_idxs, test_idxs) in enumerate(gkf.split(df, df["answer"], df["vacancy_uuid"])):
    df_train, df_test = df.loc[train_idxs], df.loc[test_idxs]
    X_train, y_train = df_train[cols_to_fit+cols_embed], df_train["answer"]
    X_test, y_test = df_test[cols_to_fit+cols_embed], df_test["answer"]
    
    model1 = CatBoostClassifier(iterations=1300, learning_rate=1e-4, random_state=42)
    model1.fit(X_train, y_train, embedding_features=cols_embed, verbose=False)
    
    preds = model1.predict(X_test)
    # loss_hist.append(accuracy_score(y_test, preds))
    # loss_hist.append(precision_score(y_test, preds))
    
    print(f"fold: {i}, accuracy: {accuracy_score(y_test, preds)}")
    # print(f"fold: {i}, [model with embeddings] accuracy: {accuracy_score(y_test, preds)}, precision: {precision_score(y_test, preds)}")

    
#     model2 = CatBoostClassifier(iterations=1300, learning_rate=1e-4, random_state=42)
#     model2.fit(X_train[cols_to_fit], y_train, verbose=False)
    
#     preds = model2.predict(X_test[cols_to_fit])
#     loss_hist.append(accuracy_score(y_test, preds))
#     loss_hist.append(precision_score(y_test, preds))
    
#     print(f"fold: {i}, accuracy: {accuracy_score(y_test, preds)}")
#     print(f"fold: {i}, [model without embeddings] accuracy: {accuracy_score(y_test, preds)}, precision: {precision_score(y_test, preds)}")
    

fold: 0, accuracy: 0.7692307692307693
fold: 1, accuracy: 0.7352941176470589
fold: 2, accuracy: 0.6946564885496184
fold: 3, accuracy: 0.6230769230769231
fold: 4, accuracy: 0.7054263565891473
